In [4]:
import pdfplumber
import json
import psycopg2
import requests


In [5]:
# 📌 EXTRACTION DU TEXTE PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
    return text

In [6]:
texte = extract_text_from_pdf("C:/Users/danie/OneDrive/Documents/GitHub/Challenge_web_mining/data/lm/femme5_engineer.pdf")
print(texte)

Objet : Candidature au poste de Data Engineer
Madame, Monsieur,
Étudiante en Licence 3 MIAGE, je suis à la recherche d'un stage dans le domaine de l'ingénierie
des données. Mon parcours académique en gestion et informatique m'a permis de développer des
compétences en manipulation et structuration des bases de données.
Lors de mes expériences en analyse statistique et développement en Python, j'ai appris à concevoir
des pipelines de données et à optimiser leur traitement pour fournir des insights pertinents. Ma
maîtrise des bases de données (MySQL, SQL Server) ainsi que des outils de gestion et
transformation de données me permet d'améliorer les performances des systèmes d'information.
Je suis motivée à intégrer votre entreprise afin d'appliquer mes connaissances et d'approfondir mon
expertise en ingénierie des données. Mon sens de l'organisation et ma capacité à résoudre des
problèmes techniques feront de moi un élément clé dans votre équipe.
Je serais ravie d'échanger avec vous lors d

In [7]:
# 📌 APPEL API MISTRAL AVEC JSON STRICT
MISTRAL_API_KEY = "Yp0Uo7Vx4uSJIlc94dj3MA5ME71KpwIR"
API_URL = "https://api.mistral.ai/v1/chat/completions"

def query_mistral(prompt):
    headers = {
        "Authorization": f"Bearer {MISTRAL_API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "mistral-medium",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }

    response = requests.post(API_URL, headers=headers, json=data)

    if response.status_code == 200:
        try:
            return json.loads(response.json()["choices"][0]["message"]["content"])
        except json.JSONDecodeError:
            print("⚠️ Erreur : réponse Mistral non valide en JSON.")
            return None
    else:
        print("❌ Erreur API :", response.text)
        return None

# 📌 EXTRACTION DES DONNÉES AVEC FORMAT JSON STRICT
def extract_data(text):
    prompt = f"""
    Voici une lettre de motivation :
    "{text}"

    Formate ta réponse en JSON avec ces champs :
    {{
      "competences": ["compétence1", "compétence2"],
      "motivations": ["motivation1", "motivation2"],
      "lieu": "ville"
    }}
    """
    return query_mistral(prompt)

In [8]:
data = extract_data(texte)
print(data)

{'competences': ['Manipulation et structuration des bases de données', 'Conception de pipelines de données', 'Optimisation du traitement des données', 'Maîtrise des bases de données (MySQL, SQL Server)', 'Outils de gestion et transformation de données', "Amélioration des performances des systèmes d'information"], 'motivations': ['Appliquer mes connaissances en ingénierie des données', 'Approfondir mon expertise en ingénierie des données', "Devenir un élément clé dans l'équipe"], 'lieu': 'non spécifié dans la lettre de motivation'}


In [9]:

# 📌 INSÉRER DANS POSTGRESQL
def insert_into_db(data):
    if not data:
        print("⚠️ Pas de données à insérer.")
        return

    try:
        conn = psycopg2.connect(
            dbname="webmining",
            user="postgres",
            password="postgres",
            host="localhost",
            port="5432",
        )
        cursor = conn.cursor()
        

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS lm (
                id SERIAL PRIMARY KEY,
                competences JSON,
                motivations JSON,
                lieu TEXT
            )
        """)

        cursor.execute("""
            INSERT INTO lm (competences, motivations, lieu)
            VALUES (%s, %s, %s)
        """, (json.dumps(data["competences"]), json.dumps(data["motivations"]), data["lieu"]))

        conn.commit()
        print("✅ Données insérées avec succès !")

    except Exception as e:
        print("❌ Erreur PostgreSQL :", e)

    finally:
        cursor.close()
        conn.close()



In [11]:
# 📌 EXÉCUTION DU PIPELINE
if data:
    print("✅ Compétences :", data["competences"])
    print("✅ Motivations :", data["motivations"])
    print("✅ Lieu :", data["lieu"])
    insert_into_db(data)

✅ Compétences : ['Manipulation et structuration des bases de données', 'Conception de pipelines de données', 'Optimisation du traitement des données', 'Maîtrise des bases de données (MySQL, SQL Server)', 'Outils de gestion et transformation de données', "Amélioration des performances des systèmes d'information"]
✅ Motivations : ['Appliquer mes connaissances en ingénierie des données', 'Approfondir mon expertise en ingénierie des données', "Devenir un élément clé dans l'équipe"]
✅ Lieu : non spécifié dans la lettre de motivation
✅ Données insérées avec succès !


In [ ]:
#affichage des tables dans la bdd
import pandas as pd
conn = psycopg2.connect(
            dbname="webmining",
            user="postgres",
            password="postgres",
            host="localhost",
            port="5432",
        )
cur = conn.cursor()
query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
"""

cur.execute(query)
tables = cur.fetchall()

# Afficher les tables sous forme de DataFrame
df_tables = pd.DataFrame(tables, columns=["Table Name"])
print(df_tables)


          Table Name
0             postes
1  poste_competences
2        competences
3      poste_savoirs
4            savoirs
5                 lm


In [ ]:


# Nom de la table à afficher
table_name = "postes"

# Exécuter la requête
query = f"SELECT * FROM {table_name} LIMIT 5;"
df = pd.read_sql(query, conn)

# Afficher le DataFrame
print(df)


Empty DataFrame
Columns: [id, intitule_poste, description, experience, langue, permis, divers, reference]
Index: []


C:\Users\danie\AppData\Local\Temp\ipykernel_28180\492081478.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


### embedding


In [ ]:
from sentence_transformers import SentenceTransformer

def get_competence_embeddings(competences):

    # Charger le modèle
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    
    # Obtenir les embeddings
    embeddings = model.encode(competences)
    
    # Retourner un dictionnaire des embeddings
    return {competence: embedding for competence, embedding in zip(competences, embeddings)}


In [ ]:

embeddings_dict = get_competence_embeddings(data["competences"])
for competence, embedding in embeddings_dict.items():
    print(f"Embeddings de '{competence}': {embedding}")

Embeddings de 'Manipulation et structuration des bases de données': [-4.15274128e-02  6.02903366e-01  6.59989268e-02 -1.22140478e-02
 -2.50089735e-01 -1.66390270e-01  5.80560148e-01 -1.62549108e-01
  3.62759650e-01  2.41822109e-01  5.69272101e-01 -1.02080546e-01
  1.72695160e-01  6.26630010e-03  1.32040352e-01  1.71874598e-01
 -1.57213256e-01  6.64916158e-01 -3.57598215e-01  4.81656015e-01
  9.57715869e-01  3.67541254e-01 -4.34257202e-02 -1.35516986e-01
  3.60304974e-02  3.06842297e-01 -1.30387902e-01 -4.90748882e-02
  3.32453400e-01 -3.83031368e-03  2.17308164e-01  2.90434122e-01
  2.23966047e-01  5.38591802e-01 -1.11993089e-01 -2.99674690e-01
  1.89954601e-02 -5.39292097e-02  4.91386443e-01  6.12737119e-01
 -6.09195352e-01  2.94249028e-01 -3.79540712e-01 -1.70415998e-01
  2.18162879e-01  1.47497252e-01  9.57009345e-02  2.68143475e-01
 -2.20822141e-01 -3.41533422e-01 -3.74823779e-01  1.70671090e-01
 -4.31883097e-01 -9.51873288e-02  2.55991697e-01 -2.78797567e-01
  1.65605471e-01 -8.67